<a href="https://colab.research.google.com/github/FedericoRaschiatore0123/GATr_Deep_Learning_Project/blob/main/GA_sample_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import h5py
import os
import pandas as pd
import seaborn as sns

In [ ]:
# create a dictionary of global variables
global_variables = {

    'dim_GA': 16
}

### Geometric Algebra Embedding

Geometric algebra G_{3,0,1} is characterized by 4 basic elements (e_{0}, e_{1}, e_{2}, e_{3}) that represent planes in the 3D space. These 4 basic elements can be combined through linear combination and geometric transformation to represent geometric objects of different dimensions (planes, lines, points), and transformations of different types (reflections, translations, rotations).

G_{3,0,1} indicates that the basis is composed by three elements (e_{1}, e_{2}, e_{3}) that satisfy e_{i}e_{i} = 1 and one (e_{0}) that satisfies e_{0}e_{0} = 0.

The basis components of the multivector associated to the G_{3,0,1} algebra are obtained by considering every possible multiplicative combination of the basis elements. Elements of different grade are thus obtained:

Grade 0 (1): scalars in the G.A. \\
Grade 1 (e_{0}, e_{i}): vectors in the G.A. \\
Grade 2 (e_{0i}, e_{ij}); bivectors in the G.A. \\
Grade 3 (e_{0ij}, e_{ijk}): trivectors in the G.A. \\
Grade 4 (e_{0123}): pseudoscalars in the G.A. \\

The multivector representation associated to the Geometric Algebra G_{3,0,1} will thus be composed of a total of 16 elements.

Scalars will be represented by grade 0 elements, planes by grade 1 elements, lines by grade 2 elements, points by grade 3 elements and pseudoscalar by grade 4 elements.

In [15]:
def embed_pos_mv_16(pos):
    """
    Embeds the pos torch tensor as a 16-dimensional G_{3,0,1} multivector

    Args:
        pos (torch.Tensor): tensor of input points with shape (n_elements, 3)

    Returns:
        mv torch.Tensor: tensor of multivectors of dimension 16 with shape (1, n_elements, 1, 16)
    """
    # Positions in Euclidean geometry can be embedded as points in G_{3,0,1} algebra

    # Get the number of points and the dimensionality
    n_elements = pos.shape[0]
    dim = pos.shape[1]

    # Create multivector tensor
    multivector = torch.zeros(n_elements, global_variables['dim_GA'])

    multivector[:, 14] = 1 # homogeneous component, e_{123}
    multivector[:, 11] = pos[:, 0] # x, e_{023}
    multivector[:, 12] = pos[:, 1] # y, e_{013}
    multivector[:, 13] = pos[:, 2] # z, e_{012}

    mv = multivector.reshape(1,n_elements,1,global_variables['dim_GA'])

    return mv


# ???: question about sign convention
# the choice of the sign allow to give the correct geometric interpretation
#       to the components. Important to have the convention be consistent
#       for the whole embedding

In [ ]:
# check function!
pts = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15],
             [16, 17, 18], [19, 20, 21], [22, 23, 24], [25, 26, 27], [28, 29, 30]]
pts_tensor = torch.tensor(pts)

rows = pts_tensor.shape[0]
cols = pts_tensor.shape[1]
print(rows)
print(cols)
#mv = embed_pos_mv_16(pts_tensor)
#print(mv) # ok!

10
3


In [14]:
def embed_face_mv_16(face):
    """
    Embeds the face torch tensor as a 16-dimensional G_{3,0,1} multivector

    Input:
        face torch.Tensor of size (n_elements, 3)
    Output:
        mv torch.Tensor of size (1, n_elements, 1, 16)
    """

    # An oriented surface in Euclidean geometry characterized by
    #   the normal vector n to the surface itself can be embedded as an oriented
    #   plane in G_{3,0,1} algebra


    # Get the number of points and the dimensionality of the space
    n_elements = face.shape[0]
    dim = face.shape[1]

    # Create multivector tensor
    multivector = torch.zeros(n_elements, global_variables['dim_GA'])

    multivector[:, 2] = face[:, 0] # e_{1}
    multivector[:, 3] = face[:, 1] # e_{2}
    multivector[:, 4] = face[:, 2] # e_{3}

    mv = multivector.reshape(1,n_elements,1,global_variables['dim_GA'])

    return mv

In [16]:
def embed_wss_mv_16(wss):
    """
    Embeds the wss vectors as a 16 dimensional G_{3,0,1} multivector
    Input:
      wss torch.Tensor of size (n_elements, 3)
    Output:
      mv torch.Tensor of size (1, n_elements, 1, 16)
    """

    # A 3D vector in the Euclidean space can be embedded as a translation in G_{3,0,1} algebra

    n_elements = wss.shape[0]
    dim = wss.shape[1]

    multivector = torch.zeros(n_elements, global_variables['dim_GA'])

    multivector[:, 0] = 1
    multivector[:, 5] = 0.5*wss[:, 0] # e_{01}
    multivector[:, 6] = 0.5*wss[:, 1] # e_{02}
    multivector[:, 7] = 0.5*wss[:, 2] # e_{03}

    mv = multivector.reshape(1,n_elements,1,global_variables['dim_GA'])

    return mv

In [25]:
def embed_inlet_mv_16(inlet):
    """
    Embeds the inlet torch tensor as a 16 dimensional G_{3,0,1} multivector
    Input:
        inlet torch.Tensor of size (n_elements, 1)
    Output:
        mv torch.Tensor of size (1, n_elements, 1, 16)
    """
    # Inlet indexes can be embedded as scalars in G_{3,0,1} algebra

    n_elements = inlet.shape[0]

    multivector = torch.zeros(n_elements, global_variables['dim_GA'])
    multivector[:, 0] = inlet[:]

    mv = multivector.reshape(1, n_elements, 1, global_variables['dim_GA'])

    return mv

In [29]:
def embed_pressure_mv_16(pressure):
  """
  Embeds the pressure torch tensor as a 16-dimensional G_{3,0,1} multivector
  Input:
    pressure torch.Tensor of size (n_elements, 1)
  Output:
    mv torch.Tensor of size (1, n_elements, 1, 16)
  """

  n_elements = pressure.shape[0]

  multivector = torch.zeros(n_elements, global_variables['dim_GA'])
  multivector[:,0] = pressure[:]

  mv = multivector.reshape(1, n_elements, 1, global_variables['dim_GA'])

  return mv

In [32]:
def embed_complete_mv_16(sample_path):
  # passa alla funzione il sample path,
  # fagli estrarre i tensori in torch,
  # fai l'embedding per ogni componente
  # e infine concatena tutti gli embedding

  # ritorna il multivector completo per il sample con path sample_path

  with h5py.File(sample_path, 'r') as f:
    # extract torch tensor for pos
    pos_data = f['pos']
    pos_torch = torch.tensor(pos_data[:])

    # extract torch tensor for face
    face_data = f['face']
    face_torch = torch.tensor(face_data[:])

    # extract torch tensor for wss
    wss_data = f['wss']
    wss_torch = torch.tensor(wss_data[:])

    # extract torch tensor for pressure
    pressure_data = f['pressure']
    pressure_torch = torch.tensor(pressure_data[:])

    # extract torch tensor for inlet_idcs
    inlet_data = f['inlet_idcs']
    inlet_torch = torch.tensor(inlet_data[:])

  # Get the multivector embeddings for each torch tensor
  pos_mv_16 = embed_pos_mv_16(pos_torch)
  face_mv_16 = embed_face_mv_16(face_torch)
  wss_mv_16 = embed_wss_mv_16(wss_torch)
  inlet_mv_16 = embed_inlet_mv_16(inlet_torch)
  pressure_mv_16 = embed_pressure_mv_16(pressure_torch)

  sample_embedding = torch.cat(
      [pos_mv_16, face_mv_16, wss_mv_16, inlet_mv_16, pressure_mv_16], dim = 3)

  return sample_embedding







In [30]:
## try on a sample

sample_path = "/content/sample_0001.hdf5"
sample = h5py.File(sample_path, 'r')
pos = sample['inlet_idcs'] # pos, wss, face, pressure, inlet_idcs
print(pos)

<HDF5 dataset "inlet_idcs": shape (966,), type "<i4">


In [33]:
sample_embedding = embed_complete_mv_16(sample_path)

RuntimeError: Sizes of tensors must match except in dimension 3. Expected size 19720 but got size 39436 for tensor number 1 in the list.

In [21]:
# Open the HDF5 file
with h5py.File(sample_path, 'r') as f:
    # Access the dataset
    wss_torch = f['wss']  # Replace 'your_dataset_name' with the actual dataset name

    # Convert the dataset to a Torch tensor
    tensor = torch.tensor(wss_torch[:])  # [:] gets all the data from the dataset

print(tensor)
embedding_wss = embed_wss_mv_16(tensor)
print(embedding_wss[0][1])


tensor([[-2.6196, -7.5756, -1.5137],
        [-2.6139, -7.4543, -1.5927],
        [-2.4789, -7.3923, -1.6124],
        ...,
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]])
tensor([[ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.3070, -3.7272, -0.7964,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])
